<a href="https://colab.research.google.com/github/anihab/tokenization/blob/main/Tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install libraries
import pandas as pd
import numpy as np

# Install Biopython 
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Resources:
# https://www.tutorialspoint.com/biopython/biopython_sequence_io_operations.htm
# https://huggingface.co/learn/nlp-course/chapter2/4?fw=pt
# https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt

In [ ]:
## Tokenizes a sequence given a fasta file and max length
from Bio import SeqIO
from google.colab import files

"""\
Read fasta file and truncate sequences to appropriate length, returns dataframe

Input:
  file -- str, path to fasta file
  max_length -- int, maximum sequence length

Returns:
  df -- dataframe, includes the > input line, start position, and sequence
""" 
def preprocess_data(file, max_length): 
  records = []
  for record in SeqIO.parse(file, 'fasta'):
    name = str(record.name)
    seq = str(record.seq).upper() 
    # Truncate sequences if longer than max_length
    while len(seq) > max_length:
      records.append(                  # add subsequence up to max_length
        {
          'name': name,
          'start': 0, # TODO
          'sequence': seq[:max_length]
        }
      )
      seq = seq[max_length:]           # sequence continuing from max_length
    records.append(
        {
          'name': name,
          'start': 0, # TODO
          'sequence': seq
        }
    )
  df = pd.DataFrame(data=records)
  return df

"""\
Read in sequences and tokens to attach labels and return dataframe

Input:
  sequences -- list, original sequences
  tokens -- list, tokenized sequences
  label -- int, 1 for phage or 0 for bacteria

Returns:
  df -- dataframe
""" 
def attach_labels(sequences, tokens, label):
  d = []
  for i in range(len(tokens)):
    d.append(
        {
          'sequence': sequences[i],
          'tokenized': tokens[i],
          'label': label
        }
    )
  df = pd.DataFrame(data=d)
  return df

"""\
Save the given dataframe to two separate csv files:
1. full_output.csv includes the name, start position, sequence, tokenized
   sequence, and label.
2. tokenized_output.csv includes the tokenized sequence and the label.

Input:
  df -- dataframe, full dataframe of tokenized sequences
""" 
def write_csv(df):
  df.to_csv('full_output.csv', encoding='utf-8', index=False)
  files.download('full_output.csv')

  tokenized = df[['tokenized', 'label']]
  tokenized.to_csv('tokenized_output.csv', encoding='utf-8', index=False)
  files.download('tokenized_output.csv')

## Different tokenization methods

"""\
Convert a sequence to codons

Input:
  seq -- str, original sequence

Returns:
  codons -- str, codons separated by space
"""
def seq2codon(seq):
  codon = [seq[i:i+3] for i in range(0,len(seq),3)]
  codons = " ".join(codon)
  return codons

"""\
Convert a sequence to kmers

Input:
  seq -- str, original sequence
  k -- int, kmer of length k

Returns:
  kmers -- str, kmers separated by space
"""
def seq2kmer(seq, k):
  kmer = [seq[i:i+k] for i in range(len(seq)+1-k)]
  kmers = " ".join(kmer)
  return kmers

#  TODO: byte pair encoding

In [ ]:
import os

MAX_TOKENS = 510

# Takes in a phage directory and bacteria directory, and runs all of the files through a tokenizer
def read_files(phage_dir, bacteria_dir):
  for filename in os.listdir(phage_dir)



"""\
Runs fasta files through tokenizer and adds the label of 1 for phage and
0 for bacteria. Then shuffles the rows in the dataframe and saves to CSV 

Input:
  phage -- str, path to phage fasta file
  bacteria -- str, path to bacteria fasta file
  method -- str, tokenization method of choice
  k -- int, length of kmer if using kmer tokenization
"""
def tokenize(phage, bacteria, method, *args, **kwargs):
  phage_seq = []
  phage_tokens = []
  bacteria_seq = []
  bacteria_tokens = []
  
  k = kwargs.get('k', None)

  if method == 'codon':
    max_length = MAX_TOKENS * 3
  elif method == 'kmer':
    max_length = MAX_TOKENS - (k - 1)

  # Process data to get sequences of appropriate length 
  phage_df = preprocess_data(phage, max_length)
  bacteria_df = preprocess_data(bacteria, max_length)

  phage_seq = phage_df["sequence"].values.tolist()
  bacteria_seq = bacteria_df["sequence"].values.tolist()

  # Tokenize according to chosen method
  for i in range(len(phage_seq)):
    if method == 'codon':
      phage_tokens.append(seq2codon(phage_seq[i]))
    elif method == 'kmer':
      phage_tokens.append(seq2kmer(phage_seq[i], k))
  phage_df["tokenized"] = phage_tokens
  phage_df["label"] = [1] * len(phage_tokens)
      
  for i in range(len(bacteria_seq)):
    if method == 'codon':
      bacteria_tokens.append(seq2codon(bacteria_seq[i]))
    elif method == 'kmer':
      bacteria_tokens.append(seq2kmer(bacteria_seq[i], k))
  bacteria_df["tokenized"] = bacteria_tokens
  bacteria_df["label"] = [0] * len(bacteria_tokens)
  
  # Shuffle and save to csv
  frames = [phage_df, bacteria_df]
  full_df = pd.concat(frames)
  full_df = full_df.sample(frac=1).reset_index(drop=True)
  write_csv(full_df)

In [ ]:
# ## OLD TESTING

# # Swap between tokenization methods (used for testing)
# def tokenize_sequences(file_path, method, *args, **kwargs):
#   sequences = []
#   tokens = []

#   k = kwargs.get('k', None)

#   if method == 'codon':
#     max_length = 512*3
#     sequences = preprocess_data(file_path, max_length)
#     for i in range(len(sequences)):
#       tokens.append(seq2codon(sequences[i]))
  
#   elif method == 'kmer':
#     max_length = 512
#     sequences = preprocess_data(file_path, max_length)
#     for i in range(len(sequences)):
#       tokens.append(seq2kmer(sequences[i], k))

#   # Get output
#   output = attach_labels(sequences, tokens, 0)
#   return output

# # Test 
# tokenize_sequences("/content/sample_data/GCF_022922415.1_ASM2292241v1_cds_from_genomic.fna", 'codon')